In [1]:
pip install duckduckgo-search groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 26.1 MB/s eta 0:00:00


In [6]:
import os
os.environ["GROQ_API_KEY"] ="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"


In [4]:
#!/usr/bin/env python
import sys
import json
import os
from duckduckgo_search import DDGS
from groq import Groq

def perform_search(query, max_results=5):
    """Perform a DuckDuckGo search for the query."""
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=max_results)
    return results

def format_search_results(results):
    """Format search results into a plain text context string."""
    formatted = ""
    for res in results:
        title = res.get("title", "No Title")
        url = res.get("href", "No URL")
        snippet = res.get("body", "No snippet available")
        formatted += f"Title: {title}\nURL: {url}\nSnippet: {snippet}\n\n"
    return formatted.strip()

def build_prompt(query, search_context):
    """Build a prompt that includes the search results as context."""
    prompt = (
        "Answer the following question using only the context provided below.\n\n"
        "Search Results:\n"
        f"{search_context}\n\n"
        "Question: " + query + "\nAnswer:"
    )
    return prompt

def call_groq(prompt):
    """Send the prompt to Groq API and return the generated answer."""
    groq_api_key = os.environ.get("GROQ_API_KEY")
    if not groq_api_key:
        raise ValueError("GROQ_API_KEY environment variable is not set.")
    client = Groq(api_key=groq_api_key)
    # Select an appropriate Groq model – adjust as needed
    model = "llama-3.3-70b-versatile"
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        max_completion_tokens=150,
        temperature=0.7,
        top_p=1,
    )
    return response.choices[0].message.content

class MCPServer:
    """A simple MCP server that integrates DuckDuckGo search and Groq API."""

    def process_request(self, request_str: str) -> str:
        # Parse the JSON request
        try:
            data = json.loads(request_str)
            query = data.get("message", "")
            if not query:
                return json.dumps({"error": "No message provided"})
        except Exception as e:
            return json.dumps({"error": f"Invalid JSON format: {str(e)}"})

        # Perform a search using DuckDuckGo
        search_results = perform_search(query)
        search_context = format_search_results(search_results)

        # Build a prompt that includes the search results
        prompt = build_prompt(query, search_context)

        # Call the Groq API with the constructed prompt
        groq_response = call_groq(prompt)

        # Build and return the final JSON response
        response = {
            "query": query,
            "search_context": search_context,
            "answer": groq_response
        }
        return json.dumps(response)

def main():
    server = MCPServer()
    # Continuously read from STDIN
    for line in sys.stdin:
        if line.strip():
            output = server.process_request(line.strip())
            print(output)
            sys.stdout.flush()

if __name__ == "__main__":
    main()


In [8]:
!pip install duckduckgo-search groq mcp-agent nest_asyncio


In [12]:
import os
import asyncio
import nest_asyncio

nest_asyncio.apply()

from duckduckgo_search import DDGS
from groq import Groq
from mcp_agent.app import MCPApp as OriginalMCPApp
from mcp_agent.agents.agent import Agent

# Define a minimal patched MCPApp that does not pass extra kwargs
class MCPAppPatched(OriginalMCPApp):
    def __init__(self):
        # Call the original constructor with no extra parameters.
        OriginalMCPApp.__init__(self)

# Define an asynchronous DuckDuckGo search tool
async def duckduckgo_search_tool(query: str) -> str:
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=5)
    formatted = ""
    for res in results:
        title = res.get("title", "No Title")
        url = res.get("href", "No URL")
        snippet = res.get("body", "No snippet available")
        formatted += f"Title: {title}\nURL: {url}\nSnippet: {snippet}\n\n"
    return formatted.strip()

# Define an asynchronous Groq chat tool for text generation
async def groq_chat_tool(prompt: str) -> str:
    groq_api_key = os.environ.get("GROQ_API_KEY")
    if not groq_api_key:
        raise ValueError("GROQ_API_KEY environment variable is not set. Please set it in this notebook.")
    client = Groq(api_key=groq_api_key)
    model = "llama-3.3-70b-versatile"
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        max_completion_tokens=150,
        temperature=0.7,
        top_p=1,
    )
    return response.choices[0].message.content

async def main():
    # Use our patched MCPApp that does not pass extra kwargs
    app = MCPAppPatched()

    # Create an agent with two tools: "search" and "generate"
    agent = Agent(
        name="search_agent",
        instruction="You are a search agent that first retrieves search results and then uses a language model to generate a final answer.",
        tools={
            "search": duckduckgo_search_tool,
            "generate": groq_chat_tool,
        }
    )

    async with app.run():
        user_query = "What is the latest news about AI breakthroughs?"

        # Step 1: Use the search tool to retrieve search results
        search_results = await agent.call_tool("search", user_query)

        # Step 2: Build a prompt that includes the search context
        combined_prompt = (
            "Answer the following question using only the context provided below.\n\n"
            "Search Results:\n" + search_results + "\n\n"
            "Question: " + user_query + "\nAnswer:"
        )

        # Step 3: Use the Groq chat tool to generate the final answer
        final_answer = await agent.call_tool("generate", combined_prompt)

        print("Final Answer:")
        print(final_answer)

if __name__ == "__main__":
    asyncio.run(main())


TypeError: object.__init__() takes exactly one argument (the instance to initialize)

In [16]:
# Install the required package (run this cell first)
!pip install duckduckgo-search

# Define a simple agent class that holds a dictionary of tools.
class SimpleAgent:
    def __init__(self, tools: dict):
        self.tools = tools

    def call_tool(self, tool_name: str, input_str: str) -> str:
        tool = self.tools.get(tool_name)
        if not tool:
            raise ValueError(f"Tool '{tool_name}' not found.")
        return tool(input_str)

# Define a synchronous DuckDuckGo search tool
def duckduckgo_search_tool(query: str) -> str:
    from duckduckgo_search import DDGS
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=5)
    formatted = ""
    for res in results:
        title = res.get("title", "No Title")
        url = res.get("href", "No URL")
        snippet = res.get("body", "No snippet available")
        formatted += f"Title: {title}\nURL: {url}\nSnippet: {snippet}\n\n"
    return formatted.strip()

def main():
    # Create an agent that only has a 'search' tool.
    agent = SimpleAgent(tools={"search": duckduckgo_search_tool})

    # Example user query.
    user_query = "Give me all information about small llms and give links also "

    # Use the agent to call the search tool.
    search_results = agent.call_tool("search", user_query)

    print("Search Results:")
    print(search_results)

if __name__ == "__main__":
    main()


Search Results:
Title: Comprehensive List of Small LLMs - E2E Networks
URL: https://www.e2enetworks.com/blog/comprehensive-list-of-small-llms-the-mini-giants-of-the-llm-world
Snippet: Similarly, small LLMs also pose ethical challenges, which needs to be addressed. These considerations can be considered not only for small LLMs, but for LLMs in general. Data Protection: Ensuring user data privacy and protection is necessary, especially when dealing with sensitive personal information. Small LLMs need to be designed with ...

Title: Here are 15+ Small LLMs that You can Run on Local Devices
URL: https://www.analyticsvidhya.com/blog/2024/04/smallest-llms-that-you-can-run-on-local-devices/
Snippet: Hugging Face Link: ALBERT. GPT-2 Small. Model Size: GPT-2 Small has around 117M parameters, significantly smaller than the larger GPT-2 models. Description: GPT-2 Small is a smaller version of the popular GPT-2 (Generative Pre-trained Transformer 2) model developed by OpenAI. While not as compact 

In [15]:
from duckduckgo_search import DDGS
from groq import Groq

# A simple synchronous agent that holds tools in a dictionary.
class SimpleAgent:
    def __init__(self, tools: dict):
        self.tools = tools

    def call_tool(self, tool_name: str, input_str: str) -> str:
        tool = self.tools.get(tool_name)
        if not tool:
            raise ValueError(f"Tool '{tool_name}' not found.")
        return tool(input_str)

# Synchronous DuckDuckGo search tool.
def duckduckgo_search_tool(query: str) -> str:
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=5)
    formatted = ""
    for res in results:
        title = res.get("title", "No Title")
        url = res.get("href", "No URL")
        snippet = res.get("body", "No snippet available")
        formatted += f"Title: {title}\nURL: {url}\nSnippet: {snippet}\n\n"
    return formatted.strip()

# Synchronous Groq chat tool for text generation.
def groq_chat_tool(prompt: str) -> str:
    groq_api_key = os.environ.get("GROQ_API_KEY")
    if not groq_api_key:
        raise ValueError("GROQ_API_KEY is not set. Please set it in your environment.")
    client = Groq(api_key=groq_api_key)
    model = "llama-3.3-70b-versatile"  # Choose an appropriate model from Groq.
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        max_completion_tokens=2048,
        temperature=0.7,
        top_p=1,
    )
    return response.choices[0].message.content

def main():
    # Create a simple agent with two tools: one for searching and one for generating.
    agent = SimpleAgent(tools={
        "search": duckduckgo_search_tool,
        "generate": groq_chat_tool,
    })

    print("Welcome to the MCP Agent!")
    print("Type 'exit' to quit.\n")
    while True:
        user_query = input("Enter your query: ").strip()
        if user_query.lower() == "exit":
            break

        # Step 1: Use the search tool to get DuckDuckGo search results.
        search_results = agent.call_tool("search", user_query)

        # Step 2: Build a prompt that includes the search results as context.
        combined_prompt = (
            "Answer the following question using only the context provided below.\n\n"
            "Search Results:\n" + search_results + "\n\n"
            "Question: " + user_query + "\nAnswer:"
        )

        # Step 3: Call the Groq chat tool with the combined prompt.
        final_answer = agent.call_tool("generate", combined_prompt)

        print("\nFinal Answer:")
        print(final_answer)
        print("\n" + "-"*40 + "\n")

if __name__ == "__main__":
    main()

Welcome to the MCP Agent!
Type 'exit' to quit.

Enter your query: i want to know latest tech on quantum computing also link it to th epage

Final Answer:
The latest developments in quantum computing include:

1. **Quantum chemistry applications**: Enabling the computation of single-point energies of large molecular systems with unprecedented accuracy, which could accelerate drug discovery and materials design. [https://www.openaccessgovernment.org/the-latest-developments-in-quantum-computing-a-transformative-frontier/187748/](https://www.openaccessgovernment.org/the-latest-developments-in-quantum-computing-a-transformative-frontier/187748/)
2. **Topological quantum processor**: An eight-qubit topological quantum processor has been unveiled by Microsoft and UC Santa Barbara researchers, marking a major step toward building a fully functional topological quantum computer. [https://scitechdaily.com/a-new-state-of-matter-just-changed-the-future-of-quantum-computing/](https://scitechdaily.c

KeyboardInterrupt: Interrupted by user

In [17]:
from duckduckgo_search import DDGS
from groq import Groq

# A simple synchronous agent that holds tools in a dictionary.
class SimpleAgent:
    def __init__(self, tools: dict):
        self.tools = tools

    def call_tool(self, tool_name: str, input_str: str) -> str:
        tool = self.tools.get(tool_name)
        if not tool:
            raise ValueError(f"Tool '{tool_name}' not found.")
        return tool(input_str)

# Synchronous DuckDuckGo search tool.
def duckduckgo_search_tool(query: str) -> str:
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=5)
    formatted = ""
    for res in results:
        title = res.get("title", "No Title")
        url = res.get("href", "No URL")
        snippet = res.get("body", "No snippet available")
        formatted += f"Title: {title}\nURL: {url}\nSnippet: {snippet}\n\n"
    return formatted.strip()

# Synchronous Groq chat tool for text generation.
def groq_chat_tool(prompt: str) -> str:
    groq_api_key = os.environ.get("GROQ_API_KEY")
    if not groq_api_key:
        raise ValueError("GROQ_API_KEY is not set. Please set it in your environment.")
    client = Groq(api_key=groq_api_key)
    model = "llama-3.3-70b-versatile"  # Choose an appropriate model from Groq.
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        max_completion_tokens=2048,
        temperature=0.7,
        top_p=1,
    )
    return response.choices[0].message.content

def main():
    # Create a simple agent with two tools: one for searching and one for generating.
    agent = SimpleAgent(tools={
        "search": duckduckgo_search_tool,
        "generate": groq_chat_tool,
    })

    print("Welcome to the MCP Agent!")
    print("Type 'exit' to quit.\n")
    while True:
        user_query = input("Enter your query: ").strip()
        if user_query.lower() == "exit":
            break

        # Step 1: Use the search tool to get DuckDuckGo search results.
        search_results = agent.call_tool("search", user_query)

        # Step 2: Build a prompt that includes the search results as context.
        combined_prompt = (
            "Answer the following question using only the context provided below.\n\n"
            "Search Results:\n" + search_results + "\n\n"
            "Question: " + user_query + "\nAnswer:"
        )

        # Step 3: Call the Groq chat tool with the combined prompt.
        final_answer = agent.call_tool("generate", combined_prompt)

        print("\nFinal Answer:")
        print(final_answer)
        print("\nSources:")
        print(search_results)
        print("\n" + "-"*40 + "\n")

if __name__ == "__main__":
    main()

Welcome to the MCP Agent!
Type 'exit' to quit.

Enter your query: quantum tech latest 2025

Final Answer:
According to the search results, 2025 is expected to be a significant year for quantum technology, with predictions of huge advances in quantum computing. The United Nations has designated 2025 as the International Year of Quantum Science and Technology, highlighting the global focus on quantum advancements. Some of the expected developments include the integration of quantum technology, quantum-safe networking, and the expansion of the quantum conversation beyond computing to other market-ready opportunities. Investors and organizations are taking notice, with Google's new Willow quantum chip adding over $100B to its market cap. Overall, 2025 is expected to be a pivotal year for the quantum technology industry.

Sources:
Title: 2025 Expert Quantum Predictions — Quantum Computing
URL: https://thequantuminsider.com/2024/12/31/2025-expert-quantum-predictions-quantum-computing/
Snippe